In [ ]:
import warnings, os
import datetime as dt
import itertools as it
from numpy import nan, inf
import numpy as np
import pandas as pd

warnings.filterwarnings('ignore')
pd.set_option('display.width',       None)
pd.set_option('display.max_columns', None)

# Load

In [ ]:
Raw = pd.read_csv('Storage/Raw.csv', parse_dates=['datetime'])
Raw.head()

In [ ]:
def QUERY_SELECT(Raw, SYMBOLS, TFRAMES):
    return Raw[Raw['symbol'].isin(SYMBOLS) & Raw['tf'].isin(TFRAMES)] .reset_index(drop=1)

Query = QUERY_SELECT(Raw, SYMBOLS=['WIN$'], TFRAMES=['D1'])
Query.head()

# Data

In [ ]:
def FORMAT_SOURCE(Query):
    WEEK_DAY = { 'Sunday':1, 'Monday':2, 'Tuesday':3, 'Wednesday':4, 'Thursday':5, 'Friday':6, 'Saturday':7 }

    Src = pd.DataFrame()
    Src[['A','Z']]                      = Query[['a','z']]
    Src[['Symbol','TF']]                = Query[['symbol','tf']]

    Src['Datetime']                     = Query['datetime']
    Src['Date']                         = Query['datetime'].dt.date
    Src['Time']                         = Query['datetime'].dt.time
    Src['Week Day']                     = Query['datetime'].dt.day_name().map(WEEK_DAY)

    Src[['Ticks','Volume']]             = Query[['tick_volume','real_volume']]
    Src[['Open','High','Low','Close']]  = Query[['open','high','low','close']]
    Src[['Price']]                      = Query[['close']]

    Src['Change']       = (Src['Close'] - Src['Open'])
    Src['Hilo']         = (Src['High']  - Src['Low'])

    Src['Sign Bull']    = Src['Change'].apply(lambda x: +1 if (x >= 0) else nan)
    Src['Sign Bear']    = Src['Change'].apply(lambda x: -1 if (x <  0) else nan)
    return Src 

Src = FORMAT_SOURCE(Query)
Src.head()

In [ ]:
def CALCULATIONS(Src):
    Calc = pd.DataFrame(Src)
    Calc['Chg Abs'] = Calc['Change'].abs()
    Calc['Chg Pos'] = Calc['Chg Abs'] * Calc['Sign Bull']
    Calc['Chg Neg'] = Calc['Chg Abs'] * Calc['Sign Bear']

    Calc['HL Pos']  = Calc['Hilo'] * Calc['Sign Bull']
    Calc['HL Neg']  = Calc['Hilo'] * Calc['Sign Bear']
    return Calc

Calc = CALCULATIONS(Src)
Calc.head()

# Stats

In [25]:
Calc.groupby('Week Day')[['Change']].describe().round(0).astype(int)

Change                                       
          count mean   std   min   25% 50%   75%   max
Week Day                                              
2           247  -12  1715 -7641  -994  84  1078  5768
3           250   13  1641 -4544 -1024  64  1015  5218
4           252  -52  1636 -4825 -1018  61   901  4619
5           249    2  1865 -5935 -1151  81  1126  5442
6           248  -76  1700 -5367 -1219 -88   962  5178

In [26]:
Calc.groupby('Week Day')[['Chg Abs']].describe().round(0).astype(int)

Chg Abs                                       
           count  mean   std min  25%   50%   75%   max
Week Day                                               
2            247  1282  1136   6  477  1058  1694  7641
3            250  1274  1030   0  506  1022  1712  5218
4            252  1275  1024   7  487   959  1831  4825
5            249  1453  1166   0  524  1151  2095  5935
6            248  1350  1032   6  526  1064  1968  5367

In [27]:
Calc.groupby('Week Day')[['Chg Pos']].describe().round(0).astype(int)

Chg Pos                                       
           count  mean   std min  25%   50%   75%   max
Week Day                                               
2            133  1179   980   6  448   987  1700  5768
3            131  1228  1011   0  362   983  1728  5218
4            128  1204   964  13  452   870  1874  4619
5            132  1372  1128   0  493  1047  1970  5442
6            120  1317  1015   6  504   974  1930  5178

In [28]:
Calc.groupby('Week Day')[['Chg Neg']].describe().round(0).astype(int)

Chg Neg                                       
           count  mean   std   min   25%   50%  75% max
Week Day                                               
2            114 -1402  1289 -7641 -1683 -1092 -622 -16
3            119 -1325  1053 -4544 -1680 -1074 -568  -6
4            124 -1348  1081 -4825 -1785 -1067 -502  -7
5            117 -1544  1205 -5935 -2126 -1231 -587 -13
6            128 -1382  1052 -5367 -1971 -1179 -586 -11

In [29]:
Calc.groupby('Week Day')[['Hilo']].describe().round(0).astype(int)

Hilo                                         
         count  mean   std  min   25%   50%   75%   max
Week Day                                               
2          247  2416  1167  593  1557  2240  3004  8487
3          250  2498  1121  655  1666  2202  3268  7864
4          252  2570  1130  687  1806  2398  3056  7753
5          249  2653  1233  701  1713  2454  3197  7278
6          248  2596  1119  805  1802  2409  3226  7479

In [30]:
Calc.groupby('Week Day')[['HL Pos']].describe().round(0).astype(int)

HL Pos                                         
          count  mean   std  min   25%   50%   75%   max
Week Day                                                
2           133  2287  1059  593  1477  2180  2875  7731
3           131  2349  1142  910  1538  2015  2954  7864
4           128  2511  1010  687  1829  2461  3002  7538
5           132  2583  1196  755  1617  2408  3068  6438
6           120  2577  1005  930  1840  2447  3236  5905

In [31]:
Calc.groupby('Week Day')[['HL Neg']].describe().round(0).astype(int)

HL Neg                                         
          count  mean   std   min   25%   50%   75%  max
Week Day                                                
2           114 -2566  1270 -8487 -3270 -2266 -1632 -747
3           119 -2661  1080 -5776 -3432 -2433 -1832 -655
4           124 -2631  1244 -7753 -3105 -2354 -1794 -875
5           117 -2731  1275 -7278 -3244 -2478 -1875 -701
6           128 -2613  1221 -7479 -3184 -2356 -1792 -805

# Snippets

In [32]:
def FORMAT_SOURCE(Query):
    WEEK_DAY = { 'Sunday':1, 'Monday':2, 'Tuesday':3, 'Wednesday':4, 'Thursday':5, 'Friday':6, 'Saturday':7 }
    Src = pd.DataFrame()
    Src['Week Day']     = Query['datetime'].dt.day_name().map(WEEK_DAY)
    Src['Change']       = (Src['Close'] - Src['Open'])
    Src['Hilo']         = (Src['High']  - Src['Low'])
    Src['Sign Bull']    = Src['Change'].apply(lambda x: +1 if (x >= 0) else nan)
    Src['Sign Bear']    = Src['Change'].apply(lambda x: -1 if (x <  0) else nan)
    return Src 


def CALCULATIONS(Src):
    Calc = pd.DataFrame(Src)
    Calc['Chg Abs'] = Calc['Change'].abs()
    Calc['Chg Pos'] = Calc['Chg Abs'] * Calc['Sign Bull']
    Calc['Chg Neg'] = Calc['Chg Abs'] * Calc['Sign Bear']

    Calc['HL Pos']  = Calc['Hilo'] * Calc['Sign Bull']
    Calc['HL Neg']  = Calc['Hilo'] * Calc['Sign Bear']
    return Calc